In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras import models
from keras.preprocessing import image
import os,shutil
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [2]:
image_array = np.load('../input/product-250-amazon/image_array_250_shape227.npy')
label_array = np.load('../input/product-250-amazon/label_array_250_shape227.npy')

In [3]:
train_images, val_images, train_labels, val_labels=train_test_split(image_array, label_array,
                                                                      test_size=0.3, random_state=42)
val_images,test_images, val_labels, test_labels=train_test_split(val_images, val_labels,
                                                                      test_size=0.33, random_state=42)

# Normalize pixel values to be between 0 and 1
train_images, test_images, val_images = train_images / 255, test_images / 255,val_images / 255
print('Number of   training samples : {}'.format(train_images.shape[0]))
print('Number of validation samples : {}'.format(val_images.shape[0]))
print('Number of       test samples : {}'.format(test_images.shape[0]))

Number of   training samples : 3498
Number of validation samples : 1005
Number of       test samples : 495


In [4]:
#Instantiate an empty model
model = Sequential(name = 'alexnet_by_Hoang')

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(227,227,3), kernel_size=(11,11), strides=(4,4), padding="valid"))
model.add(Activation('relu'))
model.add(BatchNormalization())
# Max Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
# Max Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(9216, input_shape=(227*227*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.5))

# 2nd Fully Connected Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.3))

# 3rd Fully Connected Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.2))

# Output Layer
model.add(Dense(20))
model.add(Activation('softmax'))

model.summary()

2022-10-01 16:13:25.754851: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-01 16:13:25.845797: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-01 16:13:25.846634: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Model: "alexnet_by_Hoang"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
activation (Activation)      (None, 55, 55, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
activation_1 (Activation)    (None, 27, 27, 256)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 256)  

2022-10-01 16:13:25.848326: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-01 16:13:25.848653: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-01 16:13:25.849350: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-01 16:13:25.849993: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [5]:
lr = 0.00001
epoch = 50
batch_size = 8

In [6]:
# Compile the model
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), 
              optimizer=Adam(learning_rate = lr), metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=epoch, batch_size = batch_size, validation_data=(val_images, val_labels))

2022-10-01 16:13:30.242135: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2162981304 exceeds 10% of free system memory.
2022-10-01 16:13:32.753449: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2162981304 exceeds 10% of free system memory.
2022-10-01 16:13:34.627563: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/50


2022-10-01 16:13:36.238005: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


438/438 [==============================] - ETA: 0s - loss: 3.0081 - accuracy: 0.0923

2022-10-01 16:13:53.954789: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 621439740 exceeds 10% of free system memory.
2022-10-01 16:13:54.875709: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 621439740 exceeds 10% of free system memory.


438/438 [==============================] - 22s 32ms/step - loss: 3.0081 - accuracy: 0.0923 - val_loss: 2.9033 - val_accuracy: 0.1194
Epoch 2/50
438/438 [==============================] - 11s 26ms/step - loss: 2.7987 - accuracy: 0.1592 - val_loss: 2.6840 - val_accuracy: 0.2199
Epoch 3/50
438/438 [==============================] - 12s 26ms/step - loss: 2.6470 - accuracy: 0.2073 - val_loss: 2.6310 - val_accuracy: 0.2109
Epoch 4/50
438/438 [==============================] - 11s 26ms/step - loss: 2.5229 - accuracy: 0.2427 - val_loss: 2.5217 - val_accuracy: 0.2428
Epoch 5/50
438/438 [==============================] - 12s 27ms/step - loss: 2.3849 - accuracy: 0.2859 - val_loss: 2.4539 - val_accuracy: 0.3035
Epoch 6/50
438/438 [==============================] - 12s 26ms/step - loss: 2.2601 - accuracy: 0.3173 - val_loss: 2.4533 - val_accuracy: 0.2935
Epoch 7/50
438/438 [==============================] - 12s 26ms/step - loss: 2.1346 - accuracy: 0.3634 - val_loss: 2.4691 - val_accuracy: 0.2905
Epo

In [7]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=1)

print('\nTest accuracy:', test_acc)

2022-10-01 16:23:22.552517: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 306082260 exceeds 10% of free system memory.


16/16 [==============================] - 1s 27ms/step - loss: 4.6988 - accuracy: 0.3737

Test accuracy: 0.3737373650074005
